In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from cdft.dft3d import dft_core
from numba import vectorize

kB = 1.380649e-23
NA = 6.02214076e23

device = torch.device('cuda')

plt.rcParams.update({'text.usetex':True, 
'font.family':'serif', 
'font.size':18, 
})

In [2]:
sigma = 1.0
epsilon = 1.0
parameters = {'sigma':sigma, 'epsilon':epsilon}

In [3]:
T = 0.75
bulk_density = torch.tensor([0.84],dtype=torch.float64)

system_size = np.array([10.0, 10.0, 10.0])
points = np.array([128, 128, 128])

dft = dft_core(parameters, T, system_size, points, device)

In [4]:
X,Y,Z = np.meshgrid(dft.x, dft.y, dft.z, indexing='ij')
r = np.sqrt(X**2+Y**2+Z**2+1e-16)

@vectorize
def LJ(r,sigma,epsilon):
    return 4.0*epsilon*((sigma/r)**12-(sigma/r)**6)

Vext = np.empty((points[0], points[1], points[2]), dtype=np.float64)
Vext = LJ(r,1.0,1.0)

In [5]:
dft.initial_condition(bulk_density,Vext)
dft.functional_derivative(fmt='ASWB')
dft.mu.numpy()
# dft.equilibrium_density_profile(bulk_density,fmt='ASWB',solver='fire',alpha0=0.1,dt=0.01,logoutput=True)

array([-8.30923279])

In [6]:
dft.dFres.cpu().numpy()

array([[[-8.30923279, -8.30923279, -8.30923279, ..., -8.30923279,
         -8.30923279, -8.30923279],
        [-8.30923279, -8.30923279, -8.30923279, ..., -8.30923279,
         -8.30923279, -8.30923279],
        [-8.30923279, -8.30923279, -8.30923279, ..., -8.30923279,
         -8.30923279, -8.30923279],
        ...,
        [-8.30923279, -8.30923279, -8.30923279, ..., -8.30923279,
         -8.30923279, -8.30923279],
        [-8.30923279, -8.30923279, -8.30923279, ..., -8.30923279,
         -8.30923279, -8.30923279],
        [-8.30923279, -8.30923279, -8.30923279, ..., -8.30923279,
         -8.30923279, -8.30923279]],

       [[-8.30923279, -8.30923279, -8.30923279, ..., -8.30923279,
         -8.30923279, -8.30923279],
        [-8.30923279, -8.30923279, -8.30923279, ..., -8.30923279,
         -8.30923279, -8.30923279],
        [-8.30923279, -8.30923279, -8.30923279, ..., -8.30923279,
         -8.30923279, -8.30923279],
        ...,
        [-8.30923279, -8.30923279, -8.30923279, ..., -

In [ ]:
ms = np.loadtxt('lj-radialdistribution-rho0.84-T0.75.dat')

plt.plot(dft.x, dft.rho[:,points[1]//2,points[2]//2].cpu()/bulk_density, 'k-')
plt.plot(ms[:,0], ms[:,1], 'C0o', markersize=7, markeredgewidth=1.8, mfc = 'none')
plt.xlim(0.0, 5.0)
plt.xlabel(r'$r/\sigma$')
plt.ylabel(r'$g(r)$')

In [ ]:
plt.figure()
c=plt.pcolormesh(X[:,:,points[1]//2],Y[:,:,points[1]//2],dft.rho[:,:,points[1]//2].cpu()/dft.rhob.cpu(), cmap='viridis'shading='gouraud')
plt.colorbar(c, label=r'$\rho (x,y)/ \rho_b$')
plt.xlabel(r'$x/\sigma$')
plt.ylabel(r'$y/\sigma$')